# Here is the task

Julia has asked us to evaluate the performance of a store trial which was performed in stores 77, 86 and 88.

We have chosen to complete this task in R, however, you will also find Python to be a useful tool in this piece of analytics. We have also provided an R solution template if you want some assistance in getting through this Task.

To get started, use the QVI_data dataset below or your output from task 1 and consider the monthly sales experience of each store. 

This can be broken down by:

- total sales revenue
- total number of customers
- average number of transactions per customer

Create a measure to compare different control stores to each of the trial stores to do this write a function to reduce having to re-do the analysis for each trial store. Consider using Pearson correlations or a metric such as a magnitude distance e.g. 1- (Observed distance – minimum distance)/(Maximum distance – minimum distance) as a measure.

Once you have selected your control stores, compare each trial and control pair during the trial period. You want to test if total sales are significantly different in the trial period and if so, check if the driver of change is more purchasing customers or more purchases per customers etc.

As we are in the early stages of this analysis Zilinka has asked us to submit our initial findings, so please save your code as a .pdf file and upload it to unlock the example answer.  

In [3]:
# Importing NumPy for numerical computations
import numpy as np

# Importing Pandas for data manipulation and analysis
import pandas as pd

# Importing Matplotlib for data visualization
import matplotlib.pyplot as plt

# Importing Seaborn for statistical data visualization
import seaborn as sns

# Importing Scikit-learn for machine learning tasks
from sklearn import datasets, linear_model

In [4]:
df = pd.read_csv('/Users/thomashobson/Documents/Quantium_Datasets/Task 2/QVI_data.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LYLTY_CARD_NBR    264834 non-null  int64  
 1   DATE              264834 non-null  object 
 2   STORE_NBR         264834 non-null  int64  
 3   TXN_ID            264834 non-null  int64  
 4   PROD_NBR          264834 non-null  int64  
 5   PROD_NAME         264834 non-null  object 
 6   PROD_QTY          264834 non-null  int64  
 7   TOT_SALES         264834 non-null  float64
 8   PACK_SIZE         264834 non-null  int64  
 9   BRAND             264834 non-null  object 
 10  LIFESTAGE         264834 non-null  object 
 11  PREMIUM_CUSTOMER  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


In [6]:
df.head(10)

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream
5,1005,2018-12-28,1,6,86,Cheetos Puffs 165g,1,2.8,165,CHEETOS,MIDAGE SINGLES/COUPLES,Mainstream
6,1007,2018-12-04,1,7,49,Infuzions SourCream&Herbs Veg Strws 110g,1,3.8,110,INFUZIONS,YOUNG SINGLES/COUPLES,Budget
7,1007,2018-12-05,1,8,10,RRD SR Slow Rst Pork Belly 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Budget
8,1009,2018-11-20,1,9,20,Doritos Cheese Supreme 330g,1,5.7,330,DORITOS,NEW FAMILIES,Premium
9,1010,2018-09-09,1,10,51,Doritos Mexicana 170g,2,8.8,170,DORITOS,YOUNG SINGLES/COUPLES,Mainstream


In [10]:
# Filter data for store numbers 77, 86, and 88
filtered_sales = df[df['STORE_NBR'].isin([77, 86, 88])]

# Calculate the total sales for each store
total_sales = filtered_sales.groupby('STORE_NBR')['TOT_SALES'].sum().reset_index()

# Display total sales for store numbers 77, 86, and 88
total_sales

,STORE_NBR,TOT_SALES
0,77,3040.00
1,86,10635.35
2,88,16333.25


In [13]:
# Filter data for store numbers 77, 86, and 88
filtered_sales = df[df['STORE_NBR'].isin([77, 86, 88])]

# Group by StoreNumber and LoyaltyNumber to count the number of unique customers
customer_count = filtered_sales.groupby(['STORE_NBR', 'LYLTY_CARD_NBR'])['LYLTY_CARD_NBR'].count().reset_index(name='CustomerCount')

# Display the count of customers per store number and loyalty number
customer_count

,STORE_NBR,LYLTY_CARD_NBR,CustomerCount
0,77,77000,3
1,77,77001,2
2,77,77002,2
3,77,77003,1
4,77,77004,2
...,...,...,...
1012,88,2370651,1
1013,88,2370701,1
1014,88,2370751,1
1015,88,2370961,2


In [14]:
# Aggregate to get total number of customers per store
total_customers = customer_count.groupby('STORE_NBR')['CustomerCount'].sum().reset_index(name='TotalCustomers')

# Display the total number of customers per store
total_customers

,STORE_NBR,TotalCustomers
0,77,563
1,86,1538
2,88,1873


In [19]:
# Filter data for store numbers 77, 86, and 88
filtered_sales = df[df['STORE_NBR'].isin([77, 86, 88])]

# Group by StoreNumber and LoyaltyNumber to count the number of unique customers
customer_count = filtered_sales.groupby(['STORE_NBR', 'LYLTY_CARD_NBR'])['LYLTY_CARD_NBR'].count().reset_index(name='CustomerCount')

# Aggregate to get total number of customers per store
total_customers = customer_count.groupby('STORE_NBR')['CustomerCount'].sum().reset_index(name='TotalCustomers')

# Calculate the total number of transactions per LoyaltyNumber
transactions_per_customer = filtered_sales.groupby('LYLTY_CARD_NBR')['LYLTY_CARD_NBR'].count().reset_index(name='Transactions')

# Calculate average transactions per customer
average_transactions_per_customer = transactions_per_customer['Transactions'].mean()

# Round the average transactions per customer to 2 decimal places
rounded_average_transactions = round(average_transactions_per_customer, 2)

# Display the rounded average number of transactions per customer
rounded_average_transactions

3.91